<a href="https://colab.research.google.com/github/parrisatwork/Prediction-of-Product-Sales/blob/main/Explaining_Models_with_Shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 Revisited - Part 2: Explanations

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
## Customization Options
pd.set_option("display.max_columns",100)
plt.style.use(['fivethirtyeight','seaborn-talk'])
mpl.rcParams['figure.facecolor']='white'
## additional required imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn import metrics
## fixing random for lesson generation
SEED = 321
np.random.seed(SEED)

<ipython-input-1-3ca4491f19d2>:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['fivethirtyeight','seaborn-talk'])


## Load Data

In [2]:
import joblib
loaded_joblib = joblib.load('/content/drive/MyDrive/Data/best_models.joblib')
loaded_joblib.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

dict_keys(['X_train_df', 'X_train_rf', 'y_train', 'X_test_df', 'X_test_rf', 'y_test', 'preprocessor', 'preprocessor_rf', 'LinearRegression', 'RandomForest'])

In [3]:
X_train = loaded_joblib['X_train_rf']
y_train = loaded_joblib['y_train']
X_test = loaded_joblib['X_test_rf']
y_test = loaded_joblib['y_test']
preprocessor = loaded_joblib['preprocessor_rf']
rf_reg = loaded_joblib['RandomForest']

In [4]:
X_train

,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Size_High,Outlet_Size_MISSING,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
8269,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.220,0.064142,61.2510,1998.0
7604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6.135,0.079294,111.2860,2009.0
2762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,12.150,0.028593,151.0708,2004.0
6464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,5.945,0.093009,127.8652,2004.0
4707,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,18.200,0.066285,247.2092,2004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,12.550,0.012216,162.7894,1985.0
8360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,11.500,0.014884,174.5080,1999.0
5929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,5.110,0.034945,161.1210,2002.0
4220,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.550,0.034800,157.5288,2007.0


In [5]:
print(f"Training R^2: {rf_reg.score(X_train,y_train):.3f}")
print(f"Test R^2: {rf_reg.score(X_test, y_test):.3f}")

Training R^2: 0.937
Test R^2: 0.561


## Apply SHAP

In [6]:
import shap
shap.initjs()

ModuleNotFoundError: ignored

### X_shap, y_shap Variables

In [ ]:
X_shap = shap.sample(X_train, nsamples=400, random_state=321)
X_shap

In [ ]:
y_shap = y_train.loc[X_shap.index]
y_shap

## Model Explainer

In [ ]:
explainer = shap.Explainer(rf_reg)
explainer

## SHAP Values

In [ ]:
shap_values = explainer(X_shap,y_shap)
shap_values.shape

In [ ]:
X_shap.shape

In [ ]:
shap_values[0]

## Summary Bar Plot

In [ ]:
fig, ax = plt.subplots()

shap.summary_plot(shap_values,X_shap, show=False, plot_type='bar')

ax.set_title('Summary Bar Plot - Affect on Sales')
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontsize=10)
plt.tight_layout();

fig.savefig('Data/summary_plot_bar_rf_reg.png', dpi=70, bbox_inches='tight')

### Summary Bar Plot Comparison

Default and Permutation Importances have the same top 5 features

SHAP Plot Summary differs in that:

'Item_Visibility' is ranked lower (5th instead of 3rd)
'Outlet_Type_Supermarket Type 3' takes the place of 'Item_Weight' (ranked 4th)
'Outlet_Identifier_OUT027' is ranked higher (3rd instead of 5th)

## Summary Dot Plot

In [ ]:
fig, ax = plt.subplots()

shap.summary_plot(shap_values,X_shap, show=False)

ax.set_title('Summary Dot Plot - Affect on Sales');

fig.savefig('Data/summary_plot_dot_rf_reg.png', dpi=70, bbox_inches='tight')

### Summary Dot Plot Interpretation

Interpret Top 3 Features

'Item_MRP':

As the value of the item's MRP increases (gets redder), the target value ('Item_Outlet_Sales') increases as well (moves right)
'Outlet_Type_Grocery_Store' (true/false):

Being a Grocery Store (red/true) will decrease (move left) the target value ('Item_Outlet_Sales')
Not being a Grocery Store (blue/false) will increase (move right) the target value ('Item_Outlet_Sales')
'Outlet_Identifier_OUT027' (true/false):

Being Outlet OUT027 (red/true) will increase (move right) the target value ('Item_Outlet_Sales')
Not being a Outlet OUT027 (blue/false) will only marginally affect (keep center at 0 or slightly move left) the target value ('Item_Outlet_Sales')

# Local Explanations

### Instantiating Local X_shap/y_shap

In [ ]:
X_shap_local = X_shap.reset_index(drop=True)
y_shap_local = y_shap.reset_index(drop=True)

### SHAP Values Local

In [ ]:
# Getting the shap values wth the local X_shap
shap_values_local = explainer(X_shap_local,y_shap_local)
shap_values_local.shape

### Predictions

In [ ]:
y_pred = rf_reg.predict(X_shap_local)
y_pred = pd.Series(y_pred, index=X_shap_local.index, name='Predicted')
y_pred

### Selecting Examples - Based on Max/Min Sales

## Get y_shap_local Indexes

In [ ]:
y_shap_local.agg(['min', 'max'])

In [ ]:
high_sales = y_shap_local == y_shap_local.max()
y_shap_local[high_sales]

In [ ]:
row_high = y_shap_local[high_sales].idxmax()
row_high

In [ ]:
low_sales = y_shap_local == y_shap_local.min()
y_shap_local[low_sales]

In [ ]:
row_low = y_shap_local[low_sales].idxmax()
row_low

## Get X_shap_local Filters

In [ ]:
filt_high_sales = X_shap_local.iloc[385]
filt_high_sales.tail()

In [ ]:
filt_low_sales = X_shap_local.iloc[178]
filt_low_sales.tail()

## Individual Force Plot

### High Sales

In [ ]:
y_shap_local[high_sales].idxmax()

In [ ]:
print(f"Example of a Store with High Sales:")
print(f"True Class: Sales = {y_shap_local.iloc[385]}")
print(f"Predicted Class: Sales = {y_pred.iloc[385]}")

In [ ]:
# row_high = 385
## Individual forceplot
shap.force_plot(explainer.expected_value,
                shap_values= shap_values[row_high].values,
               features=X_shap.iloc[row_high])

## Low Sales

In [ ]:
print(f"Example of a Store with Low Sales:")
print(f"True Class: Sales = {y_shap_local.iloc[178]}")
print(f"Predicted Class: Sales = {y_pred.iloc[178]}")

In [ ]:
# row_low = 178
## Individual forceplot
shap.force_plot(explainer.expected_value,
                shap_values= shap_values[row_low].values,
               features=X_shap.iloc[row_low])

## Lime Tabular Explanation

In [ ]:
lime_explainer =LimeTabularExplainer(
    training_data=X_shap_local.values,
    feature_names=X_shap_local.columns,
    mode='regression'
)
lime_explainer